In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import sys

import torch
from torch.utils.data import DataLoader

sys.path.append('..')
import zsl

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
ds = zsl.data.AwAFeaturesDataset(
    '../data/Animals_with_Attributes',
    features_type='cq-hist')

AttributeError: 'AwAFeaturesDataset' object has no attribute 'training_classes'

In [19]:
features, target, semantic = ds[0]
print(ds.classes[target])
print(features.shape)
print(list(zip(ds.attrs, semantic)))

antelope
torch.Size([2688])
[('black', -1.0), ('white', -1.0), ('blue', -1.0), ('brown', -1.0), ('gray', 12.34), ('orange', 0.0), ('red', 0.0), ('yellow', 0.0), ('patches', 16.11), ('spots', 9.19), ('stripes', 0.0), ('furry', 38.09), ('hairless', 4.44), ('toughskin', 28.55), ('big', 38.75), ('small', 5.68), ('bulbous', 17.07), ('lean', 39.99), ('flippers', 0.0), ('hands', 0.0), ('hooves', 67.08), ('pads', 7.78), ('paws', 0.0), ('longleg', 60.24), ('longneck', 16.8), ('tail', 40.59), ('chewteeth', 29.7), ('meatteeth', 5.56), ('buckteeth', 2.47), ('strainteeth', 0.0), ('horns', 87.43), ('claws', 0.0), ('tusks', 8.64), ('smelly', 9.04), ('flys', 0.0), ('hops', 9.23), ('swims', 1.23), ('tunnels', 0.0), ('walks', 54.58), ('fast', 70.86), ('slow', 3.33), ('strong', 33.56), ('weak', 8.15), ('muscle', 26.14), ('bipedal', 0.0), ('quadrapedal', 67.85), ('active', 41.19), ('inactive', 7.36), ('nocturnal', 1.11), ('hibernate', 6.94), ('agility', 62.32), ('fish', 0.0), ('meat', 4.44), ('plankton', 

In [20]:
train_ds = zsl.data.AwAFeaturesDataset(
    '../data/Animals_with_Attributes',
    features_type='cq-hist',
    load_unseen=False)

valid_ds = zsl.data.AwAFeaturesDataset(
    '../data/Animals_with_Attributes',
    features_type='cq-hist',
    load_unseen=True,
    load_only_unseen=True)

train_dl = DataLoader(
    train_ds, 
    shuffle=True, 
    batch_size=64,
    collate_fn=zsl.utils.collate_image_folder)

valid_dl = DataLoader(
    valid_ds, 
    batch_size=32,
    collate_fn=zsl.utils.collate_image_folder)

AttributeError: 'AwAFeaturesDataset' object has no attribute 'training_classes'

In [ ]:
valid_ds.valid_classes

In [21]:
semantic_unit = zsl.models.LinearSemanticUnit(
    in_features=len(train_ds.attrs),
    out_features=2048)
visual_fe = zsl.models.Identity(features.size(0))                                

In [22]:
features.shape

torch.Size([2688])

In [23]:
semantic = torch.FloatTensor(semantic)
print('Semantic repr shape:', semantic_unit(semantic.unsqueeze(0)).size())
print('Visual embedding shape:', visual_fe(features.unsqueeze(0)).size())

Semantic repr shape: torch.Size([1, 2048])
Visual embedding shape: torch.Size([1, 2688])


In [24]:
zs_model = zsl.models.ZeroShot(visual_fe, semantic_unit)
image_embed, semantic_embed = zs_model(
    features.unsqueeze(0), torch.FloatTensor(semantic).unsqueeze(0))
image_embed.size(), semantic_embed.size()

(torch.Size([1, 2688]), torch.Size([1, 2688]))

In [25]:
zs_model.to(device);

In [26]:
features, labels, semantics = next(iter(train_dl))
images_embeds, semantics_embeds = zs_model(features.to(device), 
                                           semantics.to(device))

images_embeds.size(), semantics_embeds.size()

(torch.Size([64, 2688]), torch.Size([64, 2688]))

In [27]:
parameters = [p for p in zs_model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(parameters, 1e-3, weight_decay=1e-5)

In [14]:
EPOCHS = 8

for epoch in range(EPOCHS):
    zsl.engine.train_epoch(
        model=zs_model, dl=train_dl, optimizer=optimizer, 
        epoch=epoch, print_freq=50, device=device)
    
    print('Running validation', end='')
    zsl.engine.evaluate(
        model=zs_model,
        dl=train_dl,
        class_representations=torch.FloatTensor(valid_ds.attr_matrix),
        device=device)

Epoch [0] [49/380] loss: 0.6416
Epoch [0] [99/380] loss: 0.3181
Epoch [0] [149/380] loss: 0.2117
Epoch [0] [199/380] loss: 0.1588
Epoch [0] [249/380] loss: 0.1271
Epoch [0] [299/380] loss: 0.1060
Epoch [0] [349/380] loss: 0.0910
Epoch [0] [380/380] loss: 0.0837
Running validationValidation accuracy: 0.0224 top_5_accuracy: 0.1128 loss: 0.0011
Epoch [1] [49/380] loss: 0.0011
Epoch [1] [99/380] loss: 0.0011
Epoch [1] [149/380] loss: 0.0011


KeyboardInterrupt: 